<a href="https://colab.research.google.com/github/TheNeodev/Notebook/blob/main/RVC-MAKER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <h1 align="center"><b> Ultimate RVC Maker 🎵 </b></h1>
<div align="center">




an high-quality voice conversion tool focused on experimenting and performance, based on Vietnamese-RVC (which is an RVC Fork).

<div align="center">

_____________________________________________________________
[support](https://discord.gg/aihub) | [github](https://github.com/unchCrew/RVC-MAKER.git) | [Terms of Use](https://github.com/unchCrew/RVC-MAKER/blob/main/TERMS_OF_USE.md)
_____________________________________________________________


**This project was created by [TheNeoDev](https://github.com/TheNeodev)** base on [PhamHuynhAnh16/Vietnamese-RVC](https://github.com/PhamHuynhAnh16/Vietnamese-RVC)

---

**Acknowledgments**

To all external collaborators for their special help in the following areas: Original RVC / Mainline (for making RVC), Vietnamese-RVC (for their up to date Fork used as a Base),

**Disclaimer**

By using Ultimate RVC Maker, you agree to comply with ethical and legal standards, respect intellectual property and privacy rights, avoid harmful or prohibited uses, and accept full responsibility for any outcomes, while Ultimate RVC Maker  disclaims liability and reserves the right to amend these terms.

In [ ]:


 #@title **INSTALL ULTIMATE RVC MAKER!**

import os
from ipywidgets import Button
from IPython.display import clear_output
print("👩🏻‍💻 INSTALLING...")

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

!git clone https://github.com/unchCrew/RVC-MAKER.git /content/program > /dev/null 2>&1
!pip install -r /content/program/requirements.txt  > /dev/null 2>&1
!pip install pyngrok > /dev/null 2>&1
!uv pip install --index-url https://download.pytorch.org/whl/cu128 > /dev/null 2>&1


#@markdown **💻 Installation will take about 2 minutes to complete!**
clear_output()
print("✅ Finished installing requirements!")

In [ ]:
#@title **Run webui**
import os
import shutil

%cd /content/RVC-MAKER
#@markdown **To experience all the features, use the interface :)**

#@markdown **If you know you can use the tensorboard to check for overtraining 👍**
tensorboard = False #@param {type:"boolean"}





# @markdown  ### Choose a sharing method:

import codecs
import threading
import urllib.request
import time
import ipywidgets as widgets
from IPython.display import display
import os
from pyngrok import ngrok
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

method = "gradio"  # @param ["gradio", "localtunnel", "ngrok"]


#@markdown If you selected the 'ngrok' method, obtain your auth token here: https://dashboard.ngrok.com/get-started/your-authtoken
ngrok_token = "" # @param {type:"string"}

def start_gradio():
    !python main/app/app.py --share

def start_localtunnel():
    !npm install -g localtunnel &>/dev/null
    with open('url.txt', 'w') as file:
        file.write('')
    get_ipython().system_raw('lt --port 7860 >> url.txt 2>&1 &')
    time.sleep(2)
    endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

    with open('url.txt', 'r') as file:
        tunnel_url = file.read()
        tunnel_url = tunnel_url.replace("your url is: ", "")

    print(f"Share Link: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m\n")

    password_endpoint_widget = widgets.Text(
    value=endpoint_ip,
    description='Password IP:',
    disabled=True
    )
    display(password_endpoint_widget)
    !python main/app/app.py


def start_ngrok():
    try:
        ngrok.set_auth_token(ngrok_token)
        ngrok.kill()
        tunnel = ngrok.connect(7860)
        print(f"Ngrok URL: \033[0m\033[93m{tunnel.public_url}\033[0m", end="\033[0m\n")
        !python app.py --listen
    except Exception as e:
        print(f"Error starting ngrok: {e}")

def start_app():
    if method == 'gradio':
        start_gradio()
    elif method == 'localtunnel':
        start_localtunnel()
    elif method == 'ngrok':
        start_ngrok()

if tensorboard:
  %load_ext tensorboard
  %tensorboard --logdir ./assets/logs --port=6870

if "autobackups" not in globals():
    autobackups = False

if autobackups:
    thread = threading.Thread(target=backup_files)
    thread.start()


thread_app = threading.Thread(target=start_app)
thread_app.start()


while True:
    time.sleep(5)

# Extra

Enjoy extra options that can make it easier for you to use Ultimate RVC Maker

In [ ]:
# @title Mount Drive
# @markdown Mount the files from Google Drive to the Colab.
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# @title Auto Backup
# @markdown When running it, it will be activated or deactivated previously to start up together with RVC Maker.
import os
import shutil
import time

LOGS_FOLDER = "/content/program/assets/logs/"
GOOGLE_DRIVE_PATH = "/content/drive/MyDrive/RVCBackup"

if "autobackups" not in globals():
    autobackups = False

cooldown = 15  # @param {type:"slider", min:0, max:100, step:0}
def backup_files():
    print("\nStarting backup loop...")
    last_backup_timestamps_path = os.path.join(
        LOGS_FOLDER, "last_backup_timestamps.txt"
    )
    fully_updated = False

    while True:
        try:
            updated_files = 0
            deleted_files = 0
            new_files = 0
            last_backup_timestamps = {}

            try:
                with open(last_backup_timestamps_path, "r") as f:
                    last_backup_timestamps = dict(line.strip().split(":") for line in f)
            except FileNotFoundError:
                pass

            for root, dirs, files in os.walk(LOGS_FOLDER):
                # Excluding "zips" and "mute" directories
                if "zips" in dirs:
                    dirs.remove("zips")
                if "mute" in dirs:
                    dirs.remove("mute")

                for filename in files:
                    if filename != "last_backup_timestamps.txt":
                        filepath = os.path.join(root, filename)
                        if os.path.isfile(filepath):
                            backup_filepath = os.path.join(
                                GOOGLE_DRIVE_PATH,
                                os.path.relpath(filepath, LOGS_FOLDER),
                            )
                            backup_folderpath = os.path.dirname(backup_filepath)
                            if not os.path.exists(backup_folderpath):
                                os.makedirs(backup_folderpath)
                            last_backup_timestamp = last_backup_timestamps.get(filepath)
                            current_timestamp = os.path.getmtime(filepath)
                            if (
                                last_backup_timestamp is None
                                or float(last_backup_timestamp) < current_timestamp
                            ):
                                shutil.copy2(filepath, backup_filepath)
                                last_backup_timestamps[filepath] = str(current_timestamp)
                                if last_backup_timestamp is None:
                                    new_files += 1
                                else:
                                    updated_files += 1


            for filepath in list(last_backup_timestamps.keys()):
                if not os.path.exists(filepath):
                    backup_filepath = os.path.join(
                        GOOGLE_DRIVE_PATH, os.path.relpath(filepath, LOGS_FOLDER)
                    )
                    if os.path.exists(backup_filepath):
                        os.remove(backup_filepath)
                        deleted_files += 1
                    del last_backup_timestamps[filepath]


            if updated_files > 0 or deleted_files > 0 or new_files > 0:
                print(f"Backup Complete: {new_files} new, {updated_files} updated, {deleted_files} deleted.")
                fully_updated = False
            elif not fully_updated:
                print("Files are up to date.")
                fully_updated = True

            with open(last_backup_timestamps_path, "w") as f:
                for filepath, timestamp in last_backup_timestamps.items():
                    f.write(f"{filepath}:{timestamp}\n")

            time.sleep(cooldown if fully_updated else 0.1)


        except Exception as error:
            print(f"An error occurred during backup: {error}")


if autobackups:
    autobackups = False
    print("Autobackup Disabled")
else:
    autobackups = True
    print("Autobackup Enabled")

In [ ]:
# @title Auto Backup
# @markdown When running it, it will be activated or deactivated previously to start up together with Applio.
import os
import shutil
import time

LOGS_FOLDER = "/content/program/assets/logs/"
GOOGLE_DRIVE_PATH = "/content/drive/MyDrive/RVCBackup"
if "autobackups" not in globals():
    autobackups = False

cooldown = 15  # @param {type:"slider", min:0, max:100, step:0}
def backup_files():
    print("\nStarting backup loop...")
    last_backup_timestamps_path = os.path.join(
        LOGS_FOLDER, "last_backup_timestamps.txt"
    )
    fully_updated = False

    while True:
        try:
            updated_files = 0
            deleted_files = 0
            new_files = 0
            last_backup_timestamps = {}

            try:
                with open(last_backup_timestamps_path, "r") as f:
                    last_backup_timestamps = dict(line.strip().split(":") for line in f)
            except FileNotFoundError:
                pass

            for root, dirs, files in os.walk(LOGS_FOLDER):
                # Excluding "zips" and "mute" directories
                if "zips" in dirs:
                    dirs.remove("zips")
                if "mute" in dirs:
                    dirs.remove("mute")

                for filename in files:
                    if filename != "last_backup_timestamps.txt":
                        filepath = os.path.join(root, filename)
                        if os.path.isfile(filepath):
                            backup_filepath = os.path.join(
                                GOOGLE_DRIVE_PATH,
                                os.path.relpath(filepath, LOGS_FOLDER),
                            )
                            backup_folderpath = os.path.dirname(backup_filepath)
                            if not os.path.exists(backup_folderpath):
                                os.makedirs(backup_folderpath)
                            last_backup_timestamp = last_backup_timestamps.get(filepath)
                            current_timestamp = os.path.getmtime(filepath)
                            if (
                                last_backup_timestamp is None
                                or float(last_backup_timestamp) < current_timestamp
                            ):
                                shutil.copy2(filepath, backup_filepath)
                                last_backup_timestamps[filepath] = str(current_timestamp)
                                if last_backup_timestamp is None:
                                    new_files += 1
                                else:
                                    updated_files += 1


            for filepath in list(last_backup_timestamps.keys()):
                if not os.path.exists(filepath):
                    backup_filepath = os.path.join(
                        GOOGLE_DRIVE_PATH, os.path.relpath(filepath, LOGS_FOLDER)
                    )
                    if os.path.exists(backup_filepath):
                        os.remove(backup_filepath)
                        deleted_files += 1
                    del last_backup_timestamps[filepath]


            if updated_files > 0 or deleted_files > 0 or new_files > 0:
                print(f"Backup Complete: {new_files} new, {updated_files} updated, {deleted_files} deleted.")
                fully_updated = False
            elif not fully_updated:
                print("Files are up to date.")
                fully_updated = True

            with open(last_backup_timestamps_path, "w") as f:
                for filepath, timestamp in last_backup_timestamps.items():
                    f.write(f"{filepath}:{timestamp}\n")

            time.sleep(cooldown if fully_updated else 0.1)


        except Exception as error:
            print(f"An error occurred during backup: {error}")


if autobackups:
    autobackups = False
    print("Autobackup Disabled")
else:
    autobackups = True
    print("Autobackup Enabled")

In [ ]:

#@title upload model to HF model
repo_hf = "NeoPy/TTS-G" # @param {type:"string"}
pth = "/content/RVC-MAKER/assets/weights/TTS_100e_500s.pth" # @param {type:"string"}
index = "/content/RVC-MAKER/assets/logs/TTS/added_IVF59_Flat_nprobe_1_TTS_v2.index" # @param {type:"string"}

#@markdown get token on https://huggingface.co/settings/tokens

token = "hf_" # @param {type:"string"}


import huggingface_hub
import zipfile
import os
def upload_model(repo, pth, index, token):
    """
    Upload a model to the Hugging Face Hub

    Args:
        repo: str, the name of the repository (including user/org, e.g., "username/repo_name")
        pth: str, path to the model file
        index: str, the index of the model in the repository
        token: str, the API token

    Returns:
        str, message indicating the success of the operation
    """
    readme = f"""
    # {repo}
    This is a model uploaded by RVC Maker, using [NeoDev](https://github.com/TheNeodev)'s script.
    """
    repo_name = repo.split('/')[1]
    with zipfile.ZipFile(f'{repo_name}.zip', 'w') as zipf:
        zipf.write(pth, os.path.basename(pth))
        zipf.write(index, os.path.basename(index))
        zipf.writestr('README.md', readme)

    # Corrected: Pass the repo_id directly as the first argument
    huggingface_hub.HfApi().create_repo(repo_id=repo, token=token, exist_ok=True)
    # Change 'path' to 'path_or_fileobj' as per the error and common usage in newer versions
    huggingface_hub.HfApi().upload_file(token=token, path_or_fileobj=f'{repo.split("/")[1]}.zip', repo_id=repo, path_in_repo=f'{repo.split("/")[1]}.zip')
    os.remove(f'{repo.split("/")[1]}.zip')
    return "Model uploaded successfully"


upload_model(repo_hf, pth, index, token)